In [1]:
import numpy as np
import libpysal
import spreg
import geopandas as gpd
import polars as pl
from local_functions import *

In [2]:
territorial_information = gpd.read_file("data/transformed/territorial_information_summary.geojson")

In [3]:
for c in territorial_information.columns:
    print(c)

Teritoriālā vienība
Platība (km^2)
Gads
Iedzīvotāju skaits gada sākumā
Gada vidējais iedzīvotāju skaits
Iedzīvotāju dabiskais pieaugums
Migrācijas saldo
Iedzīvotāju skaita izmaiņas - pavisam
Mēneša darba samaksas mediāna
Iedzīvotāji darbspējas vecumā
Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)
Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)
Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Izgl

In [8]:
y_col = ['Iedzīvotāju skaita izmaiņas - pavisam (%)']
x_col = [
    'Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Veselība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Vidējais attālums līdz bērnudārzam (m)',
    'Sabiedriskā transporta pieturu skaits (uz 1000 iedz.)',
    'Ekspluatācijā pieņemto dzīvokļu platība (tūkst. m2) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Vides aizsardzība (Kapitālie izdevumi) (uz 1 iedz.)'
]

columns_to_use = [
    'Teritoriālā vienība',
    'Gads',
    'geometry'
] + x_col + y_col

In [9]:
(
    pl.from_dataframe(territorial_information.drop("geometry", axis=1))
    .unpivot(
        on=y_col + x_col,
        index=['Teritoriālā vienība', 'Gads']
    )
    .filter(~pl.col("value").is_null())
    .group_by("variable")
    .agg(
        pl.col("Gads").min().alias("Min year"),
        pl.col("Gads").max().alias("Max year")
    )
    .group_by(1)
    .agg(
        pl.col("Min year").max(),
        pl.col("Max year").min()
    )
)

literal,Min year,Max year
i32,i32,i32
1,2021,2023


In [10]:
df = territorial_information[columns_to_use]
df = df[df.Gads.isin([2021, 2022, 2023])]
df = df.fillna(0)
df = df.sort_values(by=['Teritoriālā vienība', 'Gads'])

In [11]:
t = df[df.Gads==2023]
w = libpysal.weights.Queen.from_dataframe(t, ids="Teritoriālā vienība")
w.transform = 'r'

In [12]:
df[x_col]

,Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Veselība (Uzturēšanas izdevumi) (uz 1 iedz.),Vidējais attālums līdz bērnudārzam (m),Sabiedriskā transporta pieturu skaits (uz 1000 iedz.),Ekspluatācijā pieņemto dzīvokļu platība (tūkst. m2) (uz 1 iedz.),"Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi) (uz 1 iedz.)",Pašvaldības izdevumi: Vides aizsardzība (Kapitālie izdevumi) (uz 1 iedz.)
323,179.703851,650.161474,1.268260,8864.234807,4.733204,0.000068,35.861852,1.200599
366,239.781552,628.043332,3.111203,4563.650872,7.778351,0.000086,13.401859,0.720599
409,276.433264,786.987807,2.658031,5132.172963,8.497409,0.000052,7.224629,0.876097
301,165.659621,553.604574,5.547219,2718.345673,6.853762,0.000000,26.693673,15.729817
344,263.975962,619.892862,5.592538,2981.306563,15.558177,0.000015,63.190754,3.696505
...,...,...,...,...,...,...,...,...
375,115.138489,734.185335,0.000000,1854.070514,4.602625,0.001012,6.927497,0.000000
418,141.943583,827.251988,0.000000,1889.526140,4.613559,0.000663,3.962520,0.000000
338,166.908335,578.944077,19.860924,2467.440067,6.583103,0.000342,7.565482,0.000000
381,215.914204,655.583391,17.615425,1918.943311,7.395046,0.001014,3.392428,0.000000


In [13]:
Y_df = df[y_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[y_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [14]:
X_df = df[x_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[x_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [15]:
Y = Y_df.to_numpy()
X = X_df.to_numpy()

fe_lag = spreg.Panel_FE_Lag(Y, X, w, name_y = y_col, name_x=x_col, name_ds="df")

In [16]:
print(fe_lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG PANEL - FIXED EFFECTS
-----------------------------------------------------------------------
Data set            :          df
Weights matrix      :     unknown
Dependent Variable  :Iedzīvotāju skaita izmaiņas - pavisam (%)                Number of Observations:         129
Mean dependent var  :     -0.0000                Number of Variables   :           9
S.D. dependent var  :      0.0072                Degrees of Freedom    :         120
Pseudo R-squared    :      0.3890
Spatial Pseudo R-squared:  0.4088
Sigma-square ML     :       0.000                Log likelihood        :     170.737
S.E of regression   :       0.006                Akaike info criterion :    -323.474
                                                 Schwarz criterion     :    -297.736

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     